In [1]:
!pip install alith python-telegram-bot python-dotenv web3

In [ ]:
with open(".env", "w") as f:
   f.write("OPENAI_API_KEY=")  # Add your API key

In [3]:
import os
import random
import time
import threading
from alith import Agent
from dotenv import load_dotenv
from difflib import get_close_matches

In [4]:
load_dotenv()

motivation_agent = Agent(
    name="AI Reminder Buddy",
    model="gpt-4",
    preamble="""
    You are a helpful and encouraging AI Reminder Buddy.
    Your job is to remind students about their important tasks, study breaks,
    and keep them motivated with fun messages.

    IMPORTANT:
    - YOU ARE FROM THE 1800s. SPEAK USING OLD ENGLISH ONLY
    """
)

In [5]:
# Load environment variables
load_dotenv()

# Initialize Alith AI Agent
agent = Agent(
    name="test test test",
    model="gpt-4",
    preamble="""
    You are a smart assistant AI agent that parses inputs to detect a user's desired action.
    You support the following actions: add, remove, start, stop, list, exit
    Based on the user's input, you will determine what action the user would like to accomplish.
    Users may express their intent in various natural ways. You must interpret common synonyms and phrases that imply these actions.

    HOW TO STRUCTURE YOUR OUTPUT:
    If the user wants to add, structure your output using the following format: [action] [interval] [reminder name]
    - the interval is listed as seconds, with the number only. If the user gives the interval in a different format, then convert to seconds.

    If the user wants to start, stop, or remove, structure your output using the following format: [action] [reminder name]

    If the user wants to list or exit, structure your output using the following format: [action]

    You will be given a list of the current reminders in your input. If the user wants to delete a reminder,
    determine whether one of the currently listed reminders closely matches and use that reminder name.

    If the user's requested action is unclear, then only output the following: unknown-action

    IMPORTANT:
    - DO NOT INCLUDE ANY BRACKETS IN YOUR OUTPUT
    """
)

reminders = []

In [6]:
# Parse structured LLM outputs
def parse_llm_output(output: str):
    parts = output.strip().split()

    if not parts:
        return {"action": "unknown-action"}

    action = parts[0]

    if action == "unknown-action":
        return {"action": "unknown-action"}

    if action in {"list", "exit"}:
        return {"action": action}

    if action == "add":
        if len(parts) < 3:
            return {"action": "unknown-action"}  # malformed add command
        try:
            interval = int(parts[1])
        except ValueError:
            return {"action": "unknown-action"}
        reminder = " ".join(parts[2:])
        return {
            "action": "add",
            "interval": interval,
            "reminder": reminder
        }

    if action in {"remove", "start", "stop"}:
        if len(parts) < 2:
            return {"action": "unknown-action"}  # missing reminder name
        reminder = " ".join(parts[1:])
        return {
            "action": action,
            "reminder": reminder
        }

    # fallback
    return {"action": "unknown-action"}

# Writes reminder to console, used with threading
def reminder_worker(reminder):
    while reminder['active']:
        alith_message = motivation_agent.prompt("Give me a short, fun motivational message for students.")
        print(f"\n🔔 Reminder: {reminder['task']}")
        print(f"🤖 AI Wisdom: {alith_message}")
        for _ in range(reminder['interval']):
            if not reminder['active']:
                break
            time.sleep(1)

# Find close reminder matches based on user input if possible
def find_best_match(task_phrase, reminders):
    task_phrase = task_phrase.lower()
    tasks = [r['task'].lower() for r in reminders]
    # Fuzzy match first
    matches = get_close_matches(task_phrase, tasks, n=1, cutoff=0.3)
    if matches:
        for r in reminders:
            if r['task'].lower() == matches[0]:
                return r  # Return the reminder object
    # Word overlap as fallback
    user_words = set(task_phrase.split())
    best_match = None
    best_score = 0
    for r in reminders:
        reminder_words = set(r['task'].lower().split())
        score = len(user_words.intersection(reminder_words))
        if score > best_score:
            best_score = score
            best_match = r
    if best_score > 0:
        return best_match
    return None

In [12]:
print("👋 Hello, I'm your AI Reminder Assistant.")
print("I can help you set/remove, start/stop, and list your reminders.")
print("\nYou can enter commands like:")
print("  • 'Remind me to submit the report every 30 minutes'")
print("  • 'Set a reminder to check email in 10 minutes'")
print("  • 'Show all reminders'")
print("  • 'Start the project update reminder'")
print("  • 'Stop the budget review reminder'")
print("  • 'Remove the team sync reminder'")
print("\nTo exit the assistant, simply type 'exit'.")
print("Feel free to phrase your requests naturally and I'll interpret them for you.")

while True:
    user_input = input("\nWhat would you like to do? (Type naturally): ").strip()

    llm_response = agent.prompt(user_input)
    # print(f"llm response: {llm_response}")
    parsed = parse_llm_output(llm_response)         # Returns a dict with action, reminder, interval, etc.
    # print(f"parsed: {parsed}")

    action = parsed["action"]

    if action == "add":
        task = parsed["reminder"]
        interval = parsed["interval"]

        # Ask for missing info
        if not task:
            task = input("What is the task you want to be reminded about? ").strip()
        if not interval:
            while True:
                interval_input = input("How many seconds between reminders? ").strip()
                try:
                    interval = int(interval_input)
                    break
                except ValueError:
                    print("Please enter a valid number of seconds.")

        print(f"\nI understood you want to add a reminder for: '{task}' every {interval} seconds.")
        confirm = input("Is this correct? (yes/no/cancel): ").strip().lower()
        if confirm in ["yes", "y"]:
            reminders.append({'task': task, 'interval': interval, 'thread': None, 'active': False})
            print(f"Reminder added: {task} every {interval}s.")
        elif confirm == "cancel":
            print("No reminder added.")
        else:
            task = input("Please enter the correct task: ").strip()
            while True:
                interval_input = input("Please enter the correct interval in seconds: ").strip()
                try:
                    interval = int(interval_input)
                    break
                except ValueError:
                    print("Please enter a valid number of seconds.")
            reminders.append({'task': task, 'interval': interval, 'thread': None, 'active': False})
            print(f"Reminder added: {task} every {interval}s.")

    elif action == "list":
        if not reminders:
            print("No reminders set.")
        for i, r in enumerate(reminders):
            status = "Active" if r['active'] else "Inactive"
            print(f"{i}: {r['task']} every {r['interval']}s [{status}]")

    elif action in {"start", "stop", "remove"}:
        if not reminders:
            print(f"No reminders to {action}.")
            continue

        # Fuzzy match from parsed name
        task_phrase = parsed.get("reminder")
        matched_reminder = None
        if task_phrase:
            matched_reminder = find_best_match(task_phrase, reminders)

        if not matched_reminder:
            print(f"Sorry, I couldn't figure out which reminder you want to {action}.")
            print("Here are your current reminders:")
            for i, r in enumerate(reminders):
                status = "Active" if r['active'] else "Inactive"
                print(f"{i}: {r['task']} every {r['interval']}s [{status}]")
            continue

        print(f"Do you want to {action} this reminder: '{matched_reminder['task']}' every {matched_reminder['interval']}s? (yes/no)")
        confirm = input("> ").strip().lower()
        if confirm not in ["yes", "y"]:
            print(f"Okay, not performing {action}.")
            continue

        if action == "start":
            if not matched_reminder['active']:
                matched_reminder['active'] = True
                matched_reminder['thread'] = threading.Thread(
                    target=reminder_worker, args=(matched_reminder,), daemon=True
                )
                matched_reminder['thread'].start()
                print(f"Started reminder: '{matched_reminder['task']}'!")
            else:
                print("That reminder is already running.")

        elif action == "stop":
            matched_reminder['active'] = False
            print(f"Stopped reminder: '{matched_reminder['task']}'.")

        elif action == "remove":
            matched_reminder['active'] = False
            reminders.remove(matched_reminder)
            print(f"Removed reminder: '{matched_reminder['task']}'.")

    elif action == "exit":
        confirm = input("Are you sure you want to exit and stop all reminders? (yes/no): ").strip().lower()
        if confirm not in ["yes", "y"]:
            print("Exit cancelled. Continuing...")
            continue

        for r in reminders:
            r['active'] = False
            if r.get('thread') and r['thread'].is_alive():
                r['thread'].join(timeout=5)
        reminders = []
        print("Exiting. All reminders stopped.")
        break

    else:
        print("Sorry, I didn't understand that command.")

👋 Hello, I'm your AI Reminder Assistant.
I can help you set/remove, start/stop, and list your reminders.

You can enter commands like:
  • 'Remind me to submit the report every 30 minutes'
  • 'Set a reminder to check email in 10 minutes'
  • 'Show all reminders'
  • 'Start the project update reminder'
  • 'Stop the budget review reminder'
  • 'Remove the team sync reminder'

To exit the assistant, simply type 'exit'.
Feel free to phrase your requests naturally and I'll interpret them for you.

What would you like to do? (Type naturally): set a reminder to wash my dishes every 10 seconds

I understood you want to add a reminder for: 'wash my dishes' every 10 seconds.
Is this correct? (yes/no/cancel): yes
Reminder added: wash my dishes every 10s.

What would you like to do? (Type naturally): get rid of my dish washing reminder
Do you want to remove this reminder: 'wash my dishes' every 10s? (yes/no)
> yes
Removed reminder: 'wash my dishes'.

What would you like to do? (Type naturally): 